In [1]:
from pathlib import Path
from kedro.framework.context import load_context

current_dir = Path.cwd()
proj_path = current_dir.parent
context = load_context(proj_path)

c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import warnings


In [3]:
# head 等で表示した際に省略されないようにする
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", 10000)

# warningを表示しないようにする
warnings.filterwarnings('ignore')

c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df_train = catalog.load("train")
df_test = catalog.load("test")
df_stage = catalog.load("stagedata")
df_weapon = catalog.load("weapon")


players = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4"]

2020-10-04 17:36:03,624 - kedro.io.data_catalog - INFO - Loading data from `train` (CSVDataSet)...
2020-10-04 17:36:03,820 - kedro.io.data_catalog - INFO - Loading data from `test` (CSVDataSet)...
2020-10-04 17:36:03,913 - kedro.io.data_catalog - INFO - Loading data from `stagedata` (CSVDataSet)...
2020-10-04 17:36:03,916 - kedro.io.data_catalog - INFO - Loading data from `weapon` (CSVDataSet)...


In [5]:
FOLD_NUM = 10

In [6]:
onehot_cols = ['lobby-mode', 'mode', 'stage',
              ]

te_cat_cols = ['A1-mainweapon', 'A1-subweapon', 'A1-special',
              'A2-mainweapon', 'A2-subweapon', 'A2-special', 
              'A3-mainweapon', 'A3-subweapon', 'A3-special', 
              'A4-mainweapon', 'A4-subweapon', 'A4-special', 
              'B1-mainweapon', 'B1-subweapon', 'B1-special', 
              'B2-mainweapon', 'B2-subweapon', 'B2-special', 
              'B3-mainweapon', 'B3-subweapon', 'B3-special', 
              'B4-mainweapon', 'B4-subweapon', 'B4-special', 
              ]


In [7]:
def disconnect_player_cnt(data):
    temp_data = data.copy()
    temp_data = temp_data.isnull()
    data["A-disconnect"] = temp_data["A1-weapon"].astype("int") + temp_data["A2-weapon"].astype("int") + temp_data["A3-weapon"].astype("int") + temp_data["A4-weapon"].astype("int")
    data["A-connect"] = 4 - data["A-disconnect"]
    data["B-disconnect"] = temp_data["B1-weapon"].astype("int") + temp_data["B2-weapon"].astype("int") + temp_data["B3-weapon"].astype("int") + temp_data["B4-weapon"].astype("int")
    data["B-connect"] = 4 - data["B-disconnect"]
    return data
    

In [8]:
def rank_to_value(x):
    rank = {'c-': 2.0, 'c': 2.3, 'c+': 2.6,
            'b-': 3, 'b': 3.3, 'b+': 3.6,
            'a-': 4, 'a': 4.3, 'a+': 4.6,
            's': 5, 's+': 5.5, 'x': 6
            }
    if x in rank:
        x = rank[x]
    else:
        x = 0
    return x

In [9]:
def regular_rank(data):
    players = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4"]
    for index, row in data.iterrows():
        if row["lobby-mode"] == "regular":
            for player in players:
                data.at[index, f"{player}-rank"] = 1
    return data


In [10]:
def rank_average(data):
    data["A-rank-ave"] = (data["A1-rank"] + data["A2-rank"] + data["A3-rank"] + data["A4-rank"]) / data["A-connect"]
    data["B-rank-ave"] = (data["B1-rank"] + data["B2-rank"] + data["B3-rank"] + data["B4-rank"]) / data["B-connect"]
    return data

In [11]:
def OneHot_encoder(x, col_list):
    import category_encoders as ce
    ce_ohe = ce.OneHotEncoder(cols=col_list, handle_unknown='impute')
    return ce_ohe.fit_transform(x)
    

In [12]:
def target_encoder_test(train_x, train_y, test_x, cat_cols):
    for c in cat_cols:
        data_tmp = pd.DataFrame({c: train_x[c], 'target': train_y})
        target_mean = data_tmp.groupby(c)['target'].mean()
        
        test_x.loc[:, c] = test_x[c].map(target_mean)
        
    return test_x
            

In [13]:
def target_encoder(train_x, train_y, valid_x, cat_cols):
    from sklearn.model_selection import KFold
    
    for c in cat_cols:
        data_tmp = pd.DataFrame({c: train_x[c], 'target': train_y})
        target_mean = data_tmp.groupby(c)['target'].mean()
        
        valid_x.loc[:, c] = valid_x[c].map(target_mean)
        
        tmp = np.repeat(np.nan, train_x.shape[0])
        
        kf = StratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=1234)
        
        for idx_1, idx_2 in kf.split(train_x, train_y):
            target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
            tmp[idx_2] = train_x[c].iloc[idx_2].map(target_mean)
        train_x.loc[:,c] = tmp
    return train_x, valid_x
            

In [14]:
def level_average(data):
    data["A-level-ave"] = (data["A1-level"] + data["A2-level"] + data["A3-level"] + data["A4-level"]) / data["A-connect"]
    data["B-level-ave"] = (data["B1-level"] + data["B2-level"] + data["B3-level"] + data["B4-level"]) / data["B-connect"]
    return data

In [15]:
def data_augmentation(data):
    import random
    temp_data = data.copy()
    temp_data = temp_data.dropna()
    A_team = ["A1", "A2", "A3", "A4"]
    B_team = ["B1", "B2", "B3", "B4"]
    for index, row in temp_data.iterrows():
        rand_A = random.sample(A_team, 4)
        temp_data.at[index,"A1-weapon"], temp_data.at[index,"A2-weapon"], temp_data.at[index,"A3-weapon"], temp_data.at[index,"A4-weapon"] = \
            row[f"{rand_A[0]}-weapon"], row[f"{rand_A[1]}-weapon"], row[f"{rand_A[2]}-weapon"], row[f"{rand_A[3]}-weapon"]
        temp_data.at[index,"A1-rank"], temp_data.at[index,"A2-rank"], temp_data.at[index,"A3-rank"], temp_data.at[index,"A4-rank"] = \
            row[f"{rand_A[0]}-rank"], row[f"{rand_A[1]}-rank"], row[f"{rand_A[2]}-rank"], row[f"{rand_A[3]}-rank"]
        temp_data.at[index,"A1-level"], temp_data.at[index,"A2-level"], temp_data.at[index,"A3-level"], temp_data.at[index,"A4-level"] = \
            row[f"{rand_A[0]}-level"], row[f"{rand_A[1]}-level"], row[f"{rand_A[2]}-level"], row[f"{rand_A[3]}-level"]

        rand_B = random.sample(B_team, 4)
        temp_data.at[index,"B1-weapon"], temp_data.at[index,"B2-weapon"], temp_data.at[index,"B3-weapon"], temp_data.at[index,"B4-weapon"] = \
            row[f"{rand_B[0]}-weapon"], row[f"{rand_B[1]}-weapon"], row[f"{rand_B[2]}-weapon"], row[f"{rand_B[3]}-weapon"]
        temp_data.at[index,"B1-rank"], temp_data.at[index,"B2-rank"], temp_data.at[index,"B3-rank"], temp_data.at[index,"B4-rank"] = \
            row[f"{rand_B[0]}-rank"], row[f"{rand_B[1]}-rank"], row[f"{rand_B[2]}-rank"], row[f"{rand_B[3]}-rank"]
        temp_data.at[index,"B1-level"], temp_data.at[index,"B2-level"], temp_data.at[index,"B3-level"], temp_data.at[index,"B4-level"] = \
            row[f"{rand_B[0]}-level"], row[f"{rand_B[1]}-level"], row[f"{rand_B[2]}-level"], row[f"{rand_B[3]}-level"]
    data = pd.concat([data, temp_data])
    return data

In [16]:
def merge_weapon(data, weapondata):
    weapons = [p + "-weapon" for p in players]
    for weapon in weapons:
        temp_weapon_detail = data[[weapon]].join(weapondata.set_index("key"), on=weapon)
        weapon_detail = [weapon[:3] + col for col in temp_weapon_detail.columns]
        temp_weapon_detail.columns = weapon_detail
        data = pd.concat([data, temp_weapon_detail], axis=1)
        data = data.drop(weapon[:3] + weapon, axis=1)
        data = data.drop(weapon[:3] + 'i', axis=1)
        data = data.drop(weapon, axis=1)
    return data

In [17]:
def range_average(data):
    data["A-range-ave"] = (data["A1-range"] + data["A2-range"] + data["A3-range"] + data["A4-range"]) / data["A-connect"]
    data["B-range-ave"] = (data["B1-range"] + data["B2-range"] + data["B3-range"] + data["B4-range"]) / data["B-connect"]
    return data

In [18]:
def damage_average(data):
    data["A-damage-ave"] = (data["A1-damage"] + data["A2-damage"] + data["A3-damage"] + data["A4-damage"]) / data["A-connect"]
    data["B-damage-ave"] = (data["B1-damage"] + data["B2-damage"] + data["B3-damage"] + data["B4-damage"]) / data["B-connect"]
    return data

In [19]:
def rate_average(data):
    data["A-rate-ave"] = (data["A1-rate"] + data["A2-rate"] + data["A3-rate"] + data["A4-rate"]) / data["A-connect"]
    data["B-rate-ave"] = (data["B1-rate"] + data["B2-rate"] + data["B3-rate"] + data["B4-rate"]) / data["B-connect"]
    return data

In [20]:
def ability_to_value(data):
    players = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4"]
    for player in players:
        data[f"{player}-ability"] = data[f"{player}-level"] * data[f"{player}-rank"]
    return data

In [21]:
def ability_average(data):
    data["A-ability-ave"] = (data["A1-ability"] + data["A2-ability"] + data["A3-ability"] + data["A4-ability"]) / data["A-connect"]
    data["B-ability-ave"] = (data["B1-ability"] + data["B2-ability"] + data["B3-ability"] + data["B4-ability"]) / data["B-connect"]
    return data    

In [22]:
def category_count(data):
    teams = ["A", "B"]
    A_team = ["A1", "A2", "A3", "A4"]
    B_team = ["B1", "B2", "B3", "B4"]
    for team in teams:
        data[f"{team}-shooter"] = 0
        data[f"{team}-blaster"] = 0
        data[f"{team}-maneuver"] = 0
        data[f"{team}-reelgun"] = 0
        data[f"{team}-roller"] = 0
        data[f"{team}-brush"] = 0
        data[f"{team}-charger"] = 0
        data[f"{team}-slosher"] = 0
        data[f"{team}-splatling"] = 0
        data[f"{team}-brella"] = 0

    category_list = ['A1-category', 'A2-category', 'A3-category', 'A4-category', 
                    'B1-category', 'B2-category', 'B3-category', 'B4-category', 
                    ]    
    temp_data = OneHot_encoder(data, category_list)
    for team in teams:
        data[f"{team}-shooter"] = temp_data[f"{team}1-category_1"] + temp_data[f"{team}2-category_1"] + temp_data[f"{team}3-category_1"] + temp_data[f"{team}4-category_1"]
        data[f"{team}-blaster"] = temp_data[f"{team}1-category_2"] + temp_data[f"{team}2-category_2"] + temp_data[f"{team}3-category_2"] + temp_data[f"{team}4-category_2"]
        data[f"{team}-maneuver"] = temp_data[f"{team}1-category_3"] + temp_data[f"{team}2-category_3"] + temp_data[f"{team}3-category_3"] + temp_data[f"{team}4-category_3"]
        data[f"{team}-reelgun"] = temp_data[f"{team}1-category_4"] + temp_data[f"{team}2-category_4"] + temp_data[f"{team}3-category_4"] + temp_data[f"{team}4-category_4"]
        data[f"{team}-roller"] = temp_data[f"{team}1-category_5"] + temp_data[f"{team}2-category_5"] + temp_data[f"{team}3-category_5"] + temp_data[f"{team}4-category_5"]
        data[f"{team}-brush"] = temp_data[f"{team}1-category_6"] + temp_data[f"{team}2-category_6"] + temp_data[f"{team}3-category_6"] + temp_data[f"{team}4-category_6"]
        data[f"{team}-charger"] = temp_data[f"{team}1-category_7"] + temp_data[f"{team}2-category_7"] + temp_data[f"{team}3-category_7"] + temp_data[f"{team}4-category_7"]
        data[f"{team}-slosher"] = temp_data[f"{team}1-category_8"] + temp_data[f"{team}2-category_8"] + temp_data[f"{team}3-category_8"] + temp_data[f"{team}4-category_8"]
        data[f"{team}-splatling"] = temp_data[f"{team}1-category_9"] + temp_data[f"{team}2-category_9"] + temp_data[f"{team}3-category_9"] + temp_data[f"{team}4-category_9"]
        data[f"{team}-brella"] = temp_data[f"{team}1-category_10"] + temp_data[f"{team}2-category_10"] + temp_data[f"{team}3-category_10"] + temp_data[f"{team}4-category_10"]
    
    return data    

In [23]:
def preprocess_weapon(weapon):
    weapon["category"] = weapon["category2"]
    weapon = weapon.drop(["category1", "category2"], axis=1)
    return weapon

In [24]:
def preprocess_stage(stagedata):
    return stagedata

In [25]:
def preprocess_data(train, test, weapon, stagedata):
    train = disconnect_player_cnt(train)
    train = data_augmentation(train)
    target = train['y']
    test = disconnect_player_cnt(test)
    data = pd.concat([train, test])
    
    # テーブルを結合
    players = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4"]
    data = data.join(stagedata.set_index("stage"), on="stage")
    data = merge_weapon(data, weapon)
    
    for player in players:
        data[f"{player}-rank"] = data[f"{player}-rank"].apply(rank_to_value)
        data[f"{player}-range"] = data[f"{player}-range"].fillna(0)
        data[f"{player}-damage"] = data[f"{player}-damage"].fillna(0)
        data[f"{player}-rate"] = data[f"{player}-rate"].fillna(0)
        data[f"{player}-level"] = data[f"{player}-level"].fillna(0)
    data = regular_rank(data)
    data = rank_average(data)
    data = level_average(data)
    data = range_average(data)
    data = damage_average(data)
    data = rate_average(data)
    data = ability_to_value(data)
    data = ability_average(data)
    data = category_count(data)
    data = OneHot_encoder(data, onehot_cols)
    return data, target

In [26]:
processed_weapon = preprocess_weapon(df_weapon)

In [27]:
processed_stage = preprocess_stage(df_stage)

In [28]:
processed_data, target = preprocess_data(df_train, df_test, processed_weapon, processed_stage)

In [29]:
def drop_column(data):
    drop_col = ["id", "period", "game-ver", "lobby",
               "A1-range", "A2-range", "A3-range", "A4-range", 
               "B1-range", "B2-range", "B3-range", "B4-range", 
               "A1-damage", "A2-damage", "A3-damage", "A4-damage", 
               "B1-damage", "B2-damage", "B3-damage", "B4-damage", 
               "A1-rate", "A2-rate", "A3-rate", "A4-rate", 
               "B1-rate", "B2-rate", "B3-rate", "B4-rate",
               "A1-level", "A2-level", "A3-level", "A4-level", 
               "B1-level", "B2-level", "B3-level", "B4-level",
               "A1-ability", "A2-ability", "A3-ability", "A4-ability", 
               "B1-ability", "B2-ability", "B3-ability", "B4-ability",
               "A1-rank", "A2-rank", "A3-rank", "A4-rank", 
               "B1-rank", "B2-rank", "B3-rank", "B4-rank",
               "A1-category", "A2-category", "A3-category", "A4-category", 
               "B1-category", "B2-category", "B3-category", "B4-category",
               ]
    data = data.drop(drop_col, axis=1)
    return data

In [30]:
def process_nan(data):
    data = data.fillna(0.0)
    return data

In [31]:
def create_master_table(data):
    master_table = drop_column(data)
    master_table = process_nan(master_table)
    train = master_table.iloc[:len(target), :]
    test = master_table.iloc[len(target):, :]
    test = test.drop('y', axis=1)
    train_x = train.drop('y', axis=1)
    train_y = train['y']
    
    test = target_encoder_test(train_x, train_y, test, te_cat_cols)

    train_x = process_nan(train_x)
    test = process_nan(test)
    return train_x, train_y, test

In [38]:
train_x, train_y, test = create_master_table(processed_data)

In [39]:
from catboost import CatBoost
from catboost import Pool


In [40]:
params = {
    # タスク設定と損失関数
    'loss_function': 'Logloss',
    # 学習ラウンド数
    'num_boost_round': 1000,
    'early_stopping_rounds': 100,
}

In [41]:

kf = StratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=1234)

scores = []

pred_cv = np.zeros(len(test.index))

num_round = 1000

for i, (tdx, vdx) in enumerate(kf.split(train_x, train_y)):
    print(f'Fold : {i}')
    # 訓練用データと検証用データに分割
    X_train, X_valid, y_train, y_valid = train_x.iloc[tdx], train_x.iloc[vdx], train_y.values[tdx], train_y.values[vdx]

    X_train, X_valid = target_encoder(X_train, y_train, X_valid, te_cat_cols)
    train_pool = Pool(X_train, y_train)
    valid_pool = Pool(X_valid, y_valid)
    
    # 学習の実行
    model = CatBoost(params)
    model.fit(train_pool, eval_set=[valid_pool],  use_best_model=True)
   
    # 検証データに対する予測値を求めて、勝敗（０　or　１）に変換
    y_pred = model.predict(valid_pool, prediction_type='Class')
    
    # accuracyスコアを計算
    score_ = accuracy_score(y_valid, y_pred)
    
    # フォールド毎の検証時のスコアを格納
    scores.append(score_)
    
    #テストデータに対する予測値を求める
#    submission = model.predict(test, prediction_type='Class')
    
    #テストデータに対する予測値をフォールド数で割って蓄積
    #(フォールド毎の予測値の平均値を求めることと同じ)
#    pred_cv += submission/FOLD_NUM

# 最終的なテストデータに対する予測値を勝敗（０　or　１）に変換
#pred_cv = np.round(pred_cv)
test = Pool(test)
pred_cv = model.predict(test, prediction_type='Class')

# 最終的なaccuracyスコアを平均値で出力
print('')
print('################################')
print('CV_score:'+ str(np.mean(scores)))

Fold : 0
Learning rate set to 0.100253
0:	learn: 0.6924536	test: 0.6925584	best: 0.6925584 (0)	total: 15.9ms	remaining: 15.9s
1:	learn: 0.6919976	test: 0.6921554	best: 0.6921554 (1)	total: 31ms	remaining: 15.4s
2:	learn: 0.6914873	test: 0.6917956	best: 0.6917956 (2)	total: 46.2ms	remaining: 15.3s
3:	learn: 0.6910343	test: 0.6914583	best: 0.6914583 (3)	total: 60.4ms	remaining: 15s
4:	learn: 0.6907123	test: 0.6911900	best: 0.6911900 (4)	total: 75.9ms	remaining: 15.1s
5:	learn: 0.6903873	test: 0.6909899	best: 0.6909899 (5)	total: 91.1ms	remaining: 15.1s
6:	learn: 0.6900749	test: 0.6908058	best: 0.6908058 (6)	total: 105ms	remaining: 14.9s
7:	learn: 0.6897234	test: 0.6905141	best: 0.6905141 (7)	total: 121ms	remaining: 15s
8:	learn: 0.6893013	test: 0.6902159	best: 0.6902159 (8)	total: 136ms	remaining: 15s
9:	learn: 0.6889772	test: 0.6899691	best: 0.6899691 (9)	total: 152ms	remaining: 15s
10:	learn: 0.6886929	test: 0.6898012	best: 0.6898012 (10)	total: 167ms	remaining: 15s
11:	learn: 0.688469

99:	learn: 0.6770988	test: 0.6844505	best: 0.6844505 (99)	total: 1.49s	remaining: 13.4s
100:	learn: 0.6769952	test: 0.6844058	best: 0.6844058 (100)	total: 1.5s	remaining: 13.4s
101:	learn: 0.6768959	test: 0.6843683	best: 0.6843683 (101)	total: 1.52s	remaining: 13.4s
102:	learn: 0.6768086	test: 0.6843651	best: 0.6843651 (102)	total: 1.53s	remaining: 13.3s
103:	learn: 0.6767430	test: 0.6843310	best: 0.6843310 (103)	total: 1.54s	remaining: 13.3s
104:	learn: 0.6766244	test: 0.6842973	best: 0.6842973 (104)	total: 1.56s	remaining: 13.3s
105:	learn: 0.6765364	test: 0.6842926	best: 0.6842926 (105)	total: 1.58s	remaining: 13.3s
106:	learn: 0.6764343	test: 0.6843284	best: 0.6842926 (105)	total: 1.59s	remaining: 13.3s
107:	learn: 0.6763371	test: 0.6843129	best: 0.6842926 (105)	total: 1.61s	remaining: 13.3s
108:	learn: 0.6762480	test: 0.6843252	best: 0.6842926 (105)	total: 1.62s	remaining: 13.3s
109:	learn: 0.6761640	test: 0.6842934	best: 0.6842926 (105)	total: 1.64s	remaining: 13.3s
110:	learn: 0

197:	learn: 0.6657138	test: 0.6828480	best: 0.6827688 (187)	total: 2.96s	remaining: 12s
198:	learn: 0.6656016	test: 0.6827599	best: 0.6827599 (198)	total: 2.97s	remaining: 12s
199:	learn: 0.6655004	test: 0.6827439	best: 0.6827439 (199)	total: 2.99s	remaining: 11.9s
200:	learn: 0.6653728	test: 0.6826958	best: 0.6826958 (200)	total: 3s	remaining: 11.9s
201:	learn: 0.6652103	test: 0.6826721	best: 0.6826721 (201)	total: 3.02s	remaining: 11.9s
202:	learn: 0.6650259	test: 0.6826724	best: 0.6826721 (201)	total: 3.03s	remaining: 11.9s
203:	learn: 0.6648982	test: 0.6826628	best: 0.6826628 (203)	total: 3.05s	remaining: 11.9s
204:	learn: 0.6647883	test: 0.6826723	best: 0.6826628 (203)	total: 3.06s	remaining: 11.9s
205:	learn: 0.6646583	test: 0.6825889	best: 0.6825889 (205)	total: 3.08s	remaining: 11.9s
206:	learn: 0.6645379	test: 0.6825735	best: 0.6825735 (206)	total: 3.09s	remaining: 11.8s
207:	learn: 0.6644494	test: 0.6825723	best: 0.6825723 (207)	total: 3.11s	remaining: 11.8s
208:	learn: 0.664

294:	learn: 0.6544093	test: 0.6807267	best: 0.6807108 (293)	total: 4.41s	remaining: 10.5s
295:	learn: 0.6543124	test: 0.6807080	best: 0.6807080 (295)	total: 4.43s	remaining: 10.5s
296:	learn: 0.6542368	test: 0.6807096	best: 0.6807080 (295)	total: 4.44s	remaining: 10.5s
297:	learn: 0.6541156	test: 0.6806890	best: 0.6806890 (297)	total: 4.46s	remaining: 10.5s
298:	learn: 0.6540017	test: 0.6807104	best: 0.6806890 (297)	total: 4.47s	remaining: 10.5s
299:	learn: 0.6538931	test: 0.6807137	best: 0.6806890 (297)	total: 4.49s	remaining: 10.5s
300:	learn: 0.6537795	test: 0.6806818	best: 0.6806818 (300)	total: 4.5s	remaining: 10.5s
301:	learn: 0.6536474	test: 0.6806543	best: 0.6806543 (301)	total: 4.52s	remaining: 10.4s
302:	learn: 0.6535396	test: 0.6806812	best: 0.6806543 (301)	total: 4.53s	remaining: 10.4s
303:	learn: 0.6534347	test: 0.6807006	best: 0.6806543 (301)	total: 4.55s	remaining: 10.4s
304:	learn: 0.6533156	test: 0.6806870	best: 0.6806543 (301)	total: 4.56s	remaining: 10.4s
305:	learn:

392:	learn: 0.6441580	test: 0.6794850	best: 0.6794829 (390)	total: 5.88s	remaining: 9.08s
393:	learn: 0.6440595	test: 0.6795066	best: 0.6794829 (390)	total: 5.89s	remaining: 9.07s
394:	learn: 0.6439433	test: 0.6793927	best: 0.6793927 (394)	total: 5.91s	remaining: 9.05s
395:	learn: 0.6438380	test: 0.6793540	best: 0.6793540 (395)	total: 5.92s	remaining: 9.04s
396:	learn: 0.6437368	test: 0.6793324	best: 0.6793324 (396)	total: 5.94s	remaining: 9.02s
397:	learn: 0.6436711	test: 0.6793403	best: 0.6793324 (396)	total: 5.95s	remaining: 9.01s
398:	learn: 0.6435726	test: 0.6793223	best: 0.6793223 (398)	total: 5.97s	remaining: 8.99s
399:	learn: 0.6434611	test: 0.6793045	best: 0.6793045 (399)	total: 5.98s	remaining: 8.97s
400:	learn: 0.6433560	test: 0.6792470	best: 0.6792470 (400)	total: 6s	remaining: 8.96s
401:	learn: 0.6432588	test: 0.6792526	best: 0.6792470 (400)	total: 6.01s	remaining: 8.95s
402:	learn: 0.6431641	test: 0.6792583	best: 0.6792470 (400)	total: 6.03s	remaining: 8.93s
403:	learn: 0

...

585:	learn: 0.6264057	test: 0.6770377	best: 0.6770069 (582)	total: 8.83s	remaining: 6.24s
586:	learn: 0.6263226	test: 0.6770261	best: 0.6770069 (582)	total: 8.84s	remaining: 6.22s
587:	learn: 0.6262423	test: 0.6770191	best: 0.6770069 (582)	total: 8.86s	remaining: 6.21s
588:	learn: 0.6261434	test: 0.6769493	best: 0.6769493 (588)	total: 8.87s	remaining: 6.19s
589:	learn: 0.6260223	test: 0.6769614	best: 0.6769493 (588)	total: 8.89s	remaining: 6.17s
590:	learn: 0.6258689	test: 0.6768886	best: 0.6768886 (590)	total: 8.9s	remaining: 6.16s
591:	learn: 0.6257874	test: 0.6768898	best: 0.6768886 (590)	total: 8.91s	remaining: 6.14s
592:	learn: 0.6257110	test: 0.6768434	best: 0.6768434 (592)	total: 8.93s	remaining: 6.13s
593:	learn: 0.6256215	test: 0.6768484	best: 0.6768434 (592)	total: 8.94s	remaining: 6.11s
594:	learn: 0.6255325	test: 0.6768048	best: 0.6768048 (594)	total: 8.96s	remaining: 6.1s
595:	learn: 0.6254271	test: 0.6768688	best: 0.6768048 (594)	total: 8.97s	remaining: 6.08s
596:	learn: 

681:	learn: 0.6179138	test: 0.6764311	best: 0.6763236 (673)	total: 10.3s	remaining: 4.8s
682:	learn: 0.6178389	test: 0.6764816	best: 0.6763236 (673)	total: 10.3s	remaining: 4.79s
683:	learn: 0.6177532	test: 0.6764691	best: 0.6763236 (673)	total: 10.3s	remaining: 4.77s
684:	learn: 0.6176319	test: 0.6764282	best: 0.6763236 (673)	total: 10.3s	remaining: 4.75s
685:	learn: 0.6175481	test: 0.6764646	best: 0.6763236 (673)	total: 10.4s	remaining: 4.74s
686:	learn: 0.6174642	test: 0.6764956	best: 0.6763236 (673)	total: 10.4s	remaining: 4.72s
687:	learn: 0.6173533	test: 0.6765179	best: 0.6763236 (673)	total: 10.4s	remaining: 4.71s
688:	learn: 0.6172612	test: 0.6765765	best: 0.6763236 (673)	total: 10.4s	remaining: 4.69s
689:	learn: 0.6171523	test: 0.6765718	best: 0.6763236 (673)	total: 10.4s	remaining: 4.68s
690:	learn: 0.6170785	test: 0.6765737	best: 0.6763236 (673)	total: 10.4s	remaining: 4.66s
691:	learn: 0.6169886	test: 0.6765349	best: 0.6763236 (673)	total: 10.4s	remaining: 4.65s
692:	learn:

779:	learn: 0.6094479	test: 0.6759924	best: 0.6759804 (778)	total: 11.8s	remaining: 3.32s
780:	learn: 0.6093688	test: 0.6760358	best: 0.6759804 (778)	total: 11.8s	remaining: 3.31s
781:	learn: 0.6092925	test: 0.6759796	best: 0.6759796 (781)	total: 11.8s	remaining: 3.29s
782:	learn: 0.6092012	test: 0.6760065	best: 0.6759796 (781)	total: 11.8s	remaining: 3.27s
783:	learn: 0.6091407	test: 0.6760272	best: 0.6759796 (781)	total: 11.8s	remaining: 3.26s
784:	learn: 0.6090685	test: 0.6760601	best: 0.6759796 (781)	total: 11.8s	remaining: 3.24s
785:	learn: 0.6089861	test: 0.6760606	best: 0.6759796 (781)	total: 11.9s	remaining: 3.23s
786:	learn: 0.6088970	test: 0.6760618	best: 0.6759796 (781)	total: 11.9s	remaining: 3.21s
787:	learn: 0.6088194	test: 0.6760586	best: 0.6759796 (781)	total: 11.9s	remaining: 3.2s
788:	learn: 0.6087367	test: 0.6760619	best: 0.6759796 (781)	total: 11.9s	remaining: 3.18s
789:	learn: 0.6086474	test: 0.6760634	best: 0.6759796 (781)	total: 11.9s	remaining: 3.17s
790:	learn:

877:	learn: 0.6011749	test: 0.6753329	best: 0.6753329 (877)	total: 13.3s	remaining: 1.84s
878:	learn: 0.6010932	test: 0.6753216	best: 0.6753216 (878)	total: 13.3s	remaining: 1.83s
879:	learn: 0.6010275	test: 0.6753322	best: 0.6753216 (878)	total: 13.3s	remaining: 1.81s
880:	learn: 0.6009329	test: 0.6753644	best: 0.6753216 (878)	total: 13.3s	remaining: 1.79s
881:	learn: 0.6008519	test: 0.6753654	best: 0.6753216 (878)	total: 13.3s	remaining: 1.78s
882:	learn: 0.6007786	test: 0.6753505	best: 0.6753216 (878)	total: 13.3s	remaining: 1.76s
883:	learn: 0.6006813	test: 0.6752916	best: 0.6752916 (883)	total: 13.3s	remaining: 1.75s
884:	learn: 0.6005821	test: 0.6753170	best: 0.6752916 (883)	total: 13.4s	remaining: 1.74s
885:	learn: 0.6004713	test: 0.6752694	best: 0.6752694 (885)	total: 13.4s	remaining: 1.72s
886:	learn: 0.6003828	test: 0.6752507	best: 0.6752507 (886)	total: 13.4s	remaining: 1.71s
887:	learn: 0.6003104	test: 0.6752714	best: 0.6752507 (886)	total: 13.4s	remaining: 1.69s
888:	learn

974:	learn: 0.5931145	test: 0.6746485	best: 0.6746485 (974)	total: 14.7s	remaining: 377ms
975:	learn: 0.5930528	test: 0.6746253	best: 0.6746253 (975)	total: 14.7s	remaining: 362ms
976:	learn: 0.5929485	test: 0.6745792	best: 0.6745792 (976)	total: 14.7s	remaining: 347ms
977:	learn: 0.5928752	test: 0.6745536	best: 0.6745536 (977)	total: 14.8s	remaining: 332ms
978:	learn: 0.5927947	test: 0.6745462	best: 0.6745462 (978)	total: 14.8s	remaining: 317ms
979:	learn: 0.5926892	test: 0.6745397	best: 0.6745397 (979)	total: 14.8s	remaining: 302ms
980:	learn: 0.5925806	test: 0.6745676	best: 0.6745397 (979)	total: 14.8s	remaining: 287ms
981:	learn: 0.5924336	test: 0.6744927	best: 0.6744927 (981)	total: 14.8s	remaining: 272ms
982:	learn: 0.5923577	test: 0.6744979	best: 0.6744927 (981)	total: 14.8s	remaining: 257ms
983:	learn: 0.5922499	test: 0.6745294	best: 0.6744927 (981)	total: 14.8s	remaining: 241ms
984:	learn: 0.5922016	test: 0.6745312	best: 0.6744927 (981)	total: 14.9s	remaining: 226ms
985:	learn

In [42]:
# 提出用ファイルを作成する
pd.DataFrame({"id": range(len(pred_cv)), "y": pred_cv }).to_csv("../data/07_model_output/catboost_submission.csv", index=False)